 ![alt text](https://i.imgur.com/1WaY7aA.png)

---



---



# Lab 2.2.3 
# *Mining Social Media on Reddit*

## The Reddit API and the PRAW Package

The Reddit API is rich and complex, with many endpoints (https://www.reddit.com/dev/api/). It includes methods for navigating its collections, which include various kinds of media as well as comments. Fortunately, the Python library PRAW reduces much of this complexity.

Reddit requires developers to create and authenticate an app before they can use the API, but the process is much less onerus than some, and does not have waiting period for approval of new developers (as of 18 August 2018).

### 1. Create a Reddit App

Go to https://www.reddit.com/prefs/apps and click "create an app".

Enter the following in the form:

- a name for your app
- select "script" radio button
- a description
- a redirect URI

(Nb. For pulling data into a data science experiment, a local port can be used for the Redirect URI; try http://127.0.0.1:1410)

![image.png](attachment:image.png)

- click "create app"
- from the form that displays, copy the following to a local text file (or to this notebook):

  - name (the name you gave to your app)
  - redirect URI
  - personal use script (this is your OAuth 2 Client ID)
  - secret (this is your OAuth 2 Secret)

### 2. Register for API Access

- follow the link at https://www.reddit.com/wiki/api and read the terms of use for Reddit API access 
- fill in the form fields at the bottom 
  - make sure to enter your new OAuth Client ID where indicated
  - your use case could be something like "Training in API usage for data science projects"
  - your platform could be something like "Jupyter Notebooks / Python"
  
- click "SUBMIT"
 
- when asked for User-Agent, enter something that fits this pattern:
  `your_os-python:your_reddit_appname:v1.0 (by /u/your_reddit_username)`

### 3. Load Python Libraries

In [1]:
import praw
import requests
import json
import pprint
from datetime import datetime, date, time

### 4. Authenticate from your Python script

You could assign your authentication details explicitly, as follows:

In [2]:
my_user_agent = 'BOBBBB'   # your user Agent string goes in here
my_client_id = 'ze9HIBPNs8AtSg'   # your Client ID string goes in here
my_client_secret = 'P1LuJX-1lIyPhZbp2iY3QpwF-8c'   # your Secret string goes in here

A better way would be to store these details externally, so they are not displayed in the notebook:

- create a file called "auth_reddit.json" in your "notebooks" directory, and save your credentials there in JSON format:

`{   "my_client_id": "your Client ID string goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;` "my_client_secret": "your Secret string goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`"my_user_agent": "your user Agent string goes in here"` <br>
`}`

Use the following code to load the credentials:  

In [9]:
def writeTOjsonfile(path, filename, data):
    filrPathNameWExt = './' + path +'./' + filename + '.json'
    with open(filrPathNameWExt, 'w') as fp:
        json.dump(data, fp)
        
        
path = './'
filename = 'auth_reddit'

data = {}

data['my_user_agent']= 'BOBBBB'
data["my_client_id"] = 'ze9HIBPNs8AtSg'
data["my_client_secret"] = 'P1LuJX-1lIyPhZbp2iY3QpwF-8c'

writeTOjsonfile(path, filename, data)

In [3]:
pwd()  # make sure your working directory is where the file is

'/Users/bob/Desktop/Datascience/DSIA-SYD-FT-201907/DSIA-SYD-FT-201907/Module 2/LABS 2'

In [10]:
path_auth = 'auth_reddit.json'
auth = json.loads(open(path_auth).read())
pp = pprint.PrettyPrinter(indent=4)
# For debugging only:
#pp.pprint(auth)

my_user_agent = auth['my_user_agent']
my_client_id = auth['my_client_id']
my_client_secret = auth['my_client_secret']

Security considerations: 
- this method only keeps your credentials invisible as long as nobody else gets access to this notebook file 
- if you wanted another user to have access to the executable notebook without divulging your credentials you should set up an OAuth 2.0 workflow to let them obtain and apply their own API tokens when using your app
- if you just want to share your analyses, you could use a separate script (which you don't share) to fetch the data and save it locally, then use a second notebook (with no API access) to load and analyse the locally stored data

### 5. Exploring the API

Here is how to connect to Reddit with read-only access:

In [11]:
reddit = praw.Reddit(client_id = my_client_id, 
                     client_secret = my_client_secret, 
                     user_agent = my_user_agent)

print('Read-only = ' + str(reddit.read_only))  # Output: True

Read-only = True


In the next cell, put the cursor after the '.' and hit the [tab] key to see the available members and methods in the response object:

In [ ]:
reddit.subreddit()

Consult the PRAW and Reddit API documentation. Print a few of the response members below:

Content in Reddit is grouped by topics called "subreddits". Content, called "submissions", is fetched by calling the `subreddit` method of the connection object (which is our `reddit` variable) with an argument that matches an actual topic. 

We also need to append a further method call to a "subinstance", such as one of the following:

- controversial
- gilded
- hot
- new
- rising
- top

One of the submission objects members is `title`. Fetch and print 10 submission titles from the 'learnpython' subreddit using one of the subinstances above:

In [12]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.title)

Ask Anything Monday - Weekly Thread
I wrote a script that downloads all your reddit saved items, and can be viewed based on subreddit. I would love feedback.
Completely stumped by interview question
Do you want to learn python? Do a side project! Here's how to find the idea
Want to make a gui application in tkinter, but don't know what to use. pack, grid, or place
How i can change the format of a video/audio file?
What exactly goes in a python portfolio?
Pandas matplotlib and seaborn instead of excel.
Predictible List Shuffling
Can somebody help me with this python program?


Now retrieve 10 authors:

In [13]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.author)

AutoModerator
freeezer98
eawfji018
emandero
a2a2a2a2a2a2a2a2a2a2
kokosxD
jcovo15
flashliquid
apv507
Hench-Mark


Note that we obtained the titles and authors from separate API calls. Can we expect these to correspond to the same submissions? If not, how could we gurantee that they do?

Why doesn't the next cell produce output?

In [14]:
submissions = reddit.subreddit('learnpython').hot(limit=10)

for submission in submissions:
    print(submission.comments)

Print two comments associated with each of these submissions:

In [15]:
submissions = reddit.subreddit('learnpython').hot(limit=10)
for submission in submissions:
    top_level_comments = list(submission.comments)
    all_comments = submission.comments.list()[:2]
    for comment in all_comments:
        print(comment.body)

I took the “Python for Everybody” course on Coursera and I am wondering what to do next to get a more solid understanding of the language. Any online classes you would recommend that build on P4E but are not too advanced?
Hi guys I'm new here, so please bear with me if I'm not exactly fallen in place with the rules and regulations. I was trying out web scraping using python selenium and chrome driver and i got stuck at this one point where I'm supposed to click this element on the screen and it keeps giving me unable to locate the element. I've tried clicking this element by xpath,classname, link text and css selector but for all it comes up as unable to locate element. Also the automated chrome tab appears to go into a limbo and goes into a complete blank screen on opening the link. Could someone please help me out, any help is deeply appreciated. Thanks in advance!!!
I'm going to save this for later.
Was curious and looked through your code and had a good little laugh when I saw one 

Referring to the API documentation, explore the submissions object and print some interesting data:

In [17]:
submissions = reddit.subreddit('sydney').hot(limit = 5)


for submision in submissions:
    print(submision.title)

Sydney Daily Random Discussion - July 12, 2019
Malcolm on the Metro
Do you have any first or second hand experience with homelessness in Sydney?
Saw this guy today near Waterloo just not giving a F
Coles and Woolies must be making a fortune from people who always forget to bring their bags when they go shopping


#### Posting to Reddit

To be able to post to your Reddit account (i.e. contribute submissions), you need to connect to the API with read/write privilege. This requires an *authorised instance*, which is obtained by including your Reddit user name and password in the connection request: 

In [ ]:
reddit = praw.Reddit(client_id='my client id',
                     client_secret='my client secret',
                     user_agent='my user agent',
                     username='my username',
                     password='my password')
print(reddit.read_only)  # Output: False

You could hide these last two credentials by adding them to your JSON file and then reading all five values at once.

>
>


>
>




---



---

> > > > > > > > > © 2019 Data Science Institute of Australia


---



---

